###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2022  by D. Koehn, T. Meier and J. Stampa, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Digitale Signalbearbeitung in der Geophysik 

## Kapitel 4 Vorbereitende Datenbearbeitung

### 4.2 Auswirkung von Datenlücken auf das Signalspektrum

Zunächst soll untersucht werden, wie sich das Spektrum eines Signals verändert, wenn fehlende Intervalle der Messung mit unterschiedlichen Methoden aufgefüllt werden.
Eine einfache Möglichkeit, um fehlende Messwerte zu ersetzen, ist, dass entsprechende Intervall auf Null zu setzen. Dies entspricht einer Multiplikation der ursprünglichen, lückenlosen Funktion $f(t)$ mit einer boxcar-Funktion. Das Ergebnis ist die Funktion $\tilde{f}(t)$, wobei $2T_e$ die Breite des fehlenden Intervalls  und $t_e$ die Mitte der Lücke ist:

\begin{equation}
  \tilde{f}(t) = f(t) \, (1-boxcar{(t-t_e,T_e)}) \, .
\end{equation}

Für das Spektrum des Signals $\tilde{F}(\omega)$ folgt:

\begin{align}
  \tilde{F}(\omega) &= F(\omega) * (\delta(\omega) - 2 T_e sinc{(T_e \omega)} e^{-i\omega t_e})\, ,\notag \\
  & = F(\omega) - F(\omega) *  2 T_e sinc{(T_e \omega)} e^{-i\omega t_e} \, . 
\end{align}

Es zeigt sich, dass das Spektrum der Wertereihe mit Datenlücken aus dem tatsächlichen Spektrum der ungestörten Wertereihe und einem Störterm besteht. Der Störterm besteht aus dem ursprünglichen Spektrum gefaltet mit einer sinc-Funktion. Insbesondere werden somit Peaks des Spektrums in die Breite verschmiert und Nebenmaxima erzeugt. Mit kleiner werdender Breite ($T_e$) des fehlenden Intervalls nähert sich das Spektrum dem ungestörten Spektrum an.

Dies wollen wir in dem folgenden Jupyter Notebook an einem einfachen Beispiel genauer untersuchen.

In [2]:
# Importiere Python Bibliotheken 
# ------------------------------
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

Neben den üblichen Python Bibliotheken `NumPy` und `Matplotlib` importieren wir diesmal auch `ipywidgets`, was uns erlaubt interaktive Notebookzellen zu erzeugen, so daß wir einfacher mit den Parametern herumspielen können. 

Basierend auf dem Python Code von Übungsaufgabe 3.4 hab ich eine kleine Funktion geschrieben, welche eine Datenlücke in eine 1000 s lange Zeitreihe mit einer Sinusfunktion einbaut. Eingabeparamter sind die Breite der Lücke $T_e$, sowie die zeitliche Mitte der Lücke $t_e$. 

In [3]:
def data_gaps(Te, te):
    
    # Define parameters
    T = 25.                  # period 1 [s]
    dt = .1                  # time sampling [s]
    L = 1000.                # length of the time series [s]

    # Define sine function ...
    omega = 2. * np.pi / T   # compute circular frequency from period
    t = np.arange(0,L+dt,dt) # compute time vector
    x = np.sin(omega*t)  # compute sine wave
    
    # ... add local high frequency signal ...
    #T1 = 15                    # period 2 [s]
    #omega1 = 2. * np.pi / T1   # compute circular frequency from period 2
    #x[1000:4000] = np.sin(omega*t[1000:4000]) + np.sin(omega1*t[1000:4000])
    
    # Add data gap 
    NTeh = (int)((Te/2) // dt) # half size of the gap [#samples]
    Nte = (int)(te // dt)      # location of the gap [#samples]
    
    # Set values inside the gap to zero
    x[Nte-NTeh:Nte+NTeh] = 0.
    
    # Fourier transform
    X = np.fft.fft(x)
    
    # Normalize by length of the time series
    N = len(x)               # length of the time series [#samples]
    X = X / (N*dt)
    
    # estimate frequencies    
    freq = np.fft.fftfreq(N, d=dt)
    
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.subplot(121)
    
    plt.plot(t, x, 'b')
    plt.plot(t[Nte],0,'ro')   # mark center of data gap
    plt.xlabel('Time (s)')
    plt.ylabel('Sin(t) (s)')
    plt.title('Time Series Sin(t)')
    
    # Plot Spectrum
    plt.subplot(122)
    
    plt.plot(freq, np.abs(X), 'b')
    plt.xlabel('Freq (Hz)')
    plt.ylabel('Amplitude |X(freq)|')
    plt.title('Amplitude spectrum')
    plt.xlim(0.01, 0.09)

Mit der oben definierten Funktion `data_gaps` können wir im nächsten Schritt einen Plot erzeugen, bei dem wir interaktiv die Parameter $T_e$ und $t_e$ variieren können.

In [4]:
interactive_plot = interactive(data_gaps, Te=(0., 1000., 25.), te=(0., 1000., 100.))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=500.0, description='Te', max=1000.0, step=25.0), FloatSlider(value=500…

Im nächsten Schritt versuchen wir die Datenlücke mit interpolierten Werten zu füllen, dazu laden wir die `SciPy` Funktion [interp1d](https://docs.scipy.org/doc/scipy-1.8.0/html-scipyorg/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d) ...

In [5]:
# Load interpolation function
from scipy.interpolate import interp1d

... entferne die Daten in der Lücke und interpoliere die fehlenden Werte ...

In [6]:
def data_gaps_interp(Te, te):
    
    # Define parameters
    T = 25.                  # period [s]
    dt = .1                  # time sampling [s]
    L = 1000.                # length of the time series [s]

    # Define sine function ...
    omega = 2. * np.pi / T   # compute circular frequency from period
    t = np.arange(0,L+dt,dt) # compute time vector
    x = np.sin(omega*t)      # compute sine wave
    
    # ... add local high frequency signal ...
    #T1 = 15                    # period 2 [s]
    #omega1 = 2. * np.pi / T1   # compute circular frequency from period 2
    #x[1000:4000] = np.sin(omega*t[1000:4000]) + np.sin(omega1*t[1000:4000])
    
    # Add data gap 
    NTeh = (int)((Te/2) // dt) # half size of the gap [#samples]
    Nte = (int)(te // dt)      # location of the gap [#samples]
    
    # Remove values inside the data gap
    xg = np.delete(x, np.arange(Nte-NTeh,Nte+NTeh))
    tg = np.delete(t, np.arange(Nte-NTeh,Nte+NTeh))    
        
    # Interpolate values in data gap
    xint = interp1d(tg, xg, 'cubic')  # create interpolation function
    x = xint(t)              # interpolate values for original time t
    
    # Fourier transform
    X = np.fft.fft(x)
    
    # Normalize by length of the time series
    N = len(x)               # length of the time series [#samples]
    X = X / (N*dt)
    
    # estimate frequencies    
    freq = np.fft.fftfreq(N, d=dt)
    
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.subplot(121)
    
    plt.plot(t, x, 'b')
    plt.plot(t[Nte],0,'ro')   # mark center of data gap
    plt.xlabel('Time (s)')
    plt.ylabel('Sin(t) (s)')
    plt.title('Time Series Sin(t)')
    
    # Plot Spectrum
    plt.subplot(122)
    
    plt.plot(freq, np.abs(X), 'b')
    plt.xlabel('Freq (Hz)')
    plt.ylabel('Amplitude |X(freq)|')
    plt.title('Amplitude spectrum')
    plt.xlim(0.01, 0.09)

In [7]:
interactive_plot = interactive(data_gaps_interp, Te=(0., 1000., 25.), te=(0., 1000., 100.))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=500.0, description='Te', max=1000.0, step=25.0), FloatSlider(value=500…

Das Ergebnis der Interpolation ist mehr oder weniger erfolgreich - eher weniger wenn die Datenlücke zu groß wird.

## Zusammenfassung:

- Die Einführung von Datenlücken führt zu Nebenmaxima im Amplitudenspektrum
- Durch Interpolation der Daten können wir die Datenlücke füllen, solange die Lücke nicht zu groß wird.
- Das Beispiel zeigt allerdings auch, daß man die interpolierten Werte mit Vorsicht interpretieren sollte.